In [1]:
import sys
sys.path.append("/s/ls4/users/grartem/RL_robots/CommandClassifier")

In [2]:
import os
import json
import numpy as np
import pandas as pd
from simpletransformers.language_representation import RepresentationModel
from simpletransformers.config.model_args import ModelArgs

In [3]:
ParsedData = []
with open("/s/ls4/users/grartem/RL_robots/CommandClassifier/models/CommandParser_tokenCLS_copy-1_ipynb_data5/checkpoint-116700-epoch-20/predict_test_one_cmd_v4_05072022_low_split.jsonlines", "r") as f:
    for line in f:
        ParsedData.append(json.loads(line))

# Dictionaries


In [246]:
with open("/s/ls4/users/grartem/RL_robots/CommandClassifier/Data/Interim/labels_names.json", "r") as f:
    attribute_names = json.load(f)

In [247]:
# Собираем словари для каждого атрибута по существующим командам
# одинаковые фразы могут относиться к разным классам. тут никуда не денешься
from collections import defaultdict
attributes_dictionaries = defaultdict(list)
attributes_list = ["action", 'direction', 'meters', 'degshours', 'object1', 'nearest', 'relation1', 'object2', 'relation2', 'object3', 'self', 'gaze']
labels_list = ["A", "D", "M", "DH", "O1", "N", "R1", "O2", "R2", "O3", "S", "G"]
dictionaries_names_list = ['action', 'direction', 'meters', 'degshours', 'object', 'nearest', 'relation', 'object', 'relation', 'object', 'self', 'gaze']
with open("../../Data/Interim/data_v5_split.jsonl", "r") as f:
    for line in f:
        line = json.loads(line)
        for lab, dictionary_name in zip(labels_list,
                                      dictionaries_names_list):
            phrase = " ".join([x for x,y in zip(line[0], line[1]) if y==lab])
            if phrase!="":
                attributes_dictionaries[dictionary_name].append((phrase, line[2][labels_list.index(lab)]))
for k in attributes_dictionaries.keys():
    attributes_dictionaries[k] = set(attributes_dictionaries[k])
for k in attributes_dictionaries.keys():
    attributes_dictionaries[k] = [[x[0] for x in attributes_dictionaries[k]], [x[1] for x in attributes_dictionaries[k]]]

In [248]:
from copy import deepcopy
attributes_dictionaries_1 = deepcopy(attributes_dictionaries)

In [253]:
# собираем словари по тем файлам, которые использовал Макс
dictionaries_folder = "/s/ls4/users/grartem/RL_robots/CommandClassifier/Data/External/dict/"
attributes_dictionaries = {}
label_to_dictname = dict(zip(["A", "D", "M", "DH", "ON", "N", "RN", "S", "G"],['action', 'direction', 'meters', 'degshours', 'object', 'nearest', 'relation', 'self', 'gaze']))
label_to_attrname = dict(zip(["A", "D", "M", "DH", "ON", "N", "RN", "S", "G"],["action", 'direction', 'meters', 'degshours', 'object1', 'nearest', 'relation1', 'self', 'gaze']))
hours_map = {
    "1hour": "30",
    "2hour": "60",
    "3hour": "90",
    "4hour": "120",
    "5hour": "150",
    "6hour": "180",
    "7hour": "210",
    "8hour": "240",
    "9hour": "270",
    "10hour": "300",
    "11hour": "330",
    "12hour": "360",
    "o1hour": "15",
    "o2hour": "45",
    "o3hour": "75",
    "o4hour": "105",
    "o5hour": "135",
    "o6hour": "165",
    "o7hour": "195",
    "o8hour": "225",
    "o9hour": "255",
    "o10hour": "285",
    "o11hour": "315",
    "o12hour": "345",
}
for phrases_list_file in os.listdir(dictionaries_folder):
    with open(dictionaries_folder + phrases_list_file, "r") as f:
        phrases = f.readlines()
    phrases_list_file = phrases_list_file.split("_")
    phrases = [x.strip() for x in phrases if x.strip()!=""]
    
    label = phrases_list_file[0]
    if label=="O":
        continue
    attr_name = "_".join(phrases_list_file[1:])    
    if attr_name in ["forward", "backward", "right", "left", "north", "south", "east", "west"]:
        attr_name = "dir_" + attr_name
    if "hour" in attr_name:
        attr_name = hours_map[attr_name]
    if attr_name in ["rotate", "move"]:
        attr_name += "_dir"
    if attr_name in ["additional_verb"]:
        continue
    if attr_name in ["focus", "focus_obj"]:
        attr_name = "has_gaze_focus_on"
    if attr_name in ["nearest"]:
        attr_name = "nearest_from_type"
    if attr_name in ["after_turn_around"]: # degshours
        attr_name = "180"
    if attr_name in ["turn_around"]: # action
        attr_name = "rotate_on" # not sure
    
    if attr_name in ["degrees", "meters"]:
        attr_name = attr_name.replace("degrees", "degshours")
        if label_to_dictname[label] not in attributes_dictionaries:
            attributes_dictionaries[label_to_dictname[label]] = [[],[]]
        for val in attribute_names[attr_name]:
            if val=="":
                continue
            for postfix in phrases:
                phrase = val+" " + postfix
        
                attributes_dictionaries[label_to_dictname[label]][0].append(phrase)
                dict_class_id = attribute_names[label_to_attrname[label]].index(val)
                attributes_dictionaries[label_to_dictname[label]][1].append(dict_class_id)
        continue
    if label_to_dictname[label] not in attributes_dictionaries:
        attributes_dictionaries[label_to_dictname[label]] = [[],[]]
    attributes_dictionaries[label_to_dictname[label]][0].extend(phrases)
    dict_class_id = attribute_names[label_to_attrname[label]].index(attr_name)
    attributes_dictionaries[label_to_dictname[label]][1].extend([dict_class_id for x in phrases])
    
    if attr_name in ["rotate_dir", "move_dir"]:
        attr_name = attr_name.replace("_dir", "_to")
        dict_class_id = attribute_names[label_to_attrname[label]].index(attr_name)
        attributes_dictionaries[label_to_dictname[label]][0].extend(phrases)
        attributes_dictionaries[label_to_dictname[label]][1].extend([dict_class_id for x in phrases])
        attr_name = attr_name.replace("_to", "_on")
        dict_class_id = attribute_names[label_to_attrname[label]].index(attr_name)
        attributes_dictionaries[label_to_dictname[label]][0].extend(phrases)
        attributes_dictionaries[label_to_dictname[label]][1].extend([dict_class_id for x in phrases])

In [254]:
# Добавить к предыдущим вариации из корпуса
attributes_list = ["action", 'direction', 'meters', 'degshours', 'object1', 'nearest', 'relation1', 'object2', 'relation2', 'object3', 'self', 'gaze']
labels_list = ["A", "D", "M", "DH", "O1", "N", "R1", "O2", "R2", "O3", "S", "G"]
dictionaries_names_list = ['action', 'direction', 'meters', 'degshours', 'object', 'nearest', 'relation', 'object', 'relation', 'object', 'self', 'gaze']
with open("../../Data/Interim/data_v5_split.jsonl", "r") as f:
    for line in f:
        line = json.loads(line)
        for lab, dictionary_name in zip(labels_list,
                                      dictionaries_names_list):
            if dictionary_name!="object":
                continue
            phrase = " ".join([x for x,y in zip(line[0], line[1]) if y==lab])
            if phrase!="":
                if phrase not in attributes_dictionaries[dictionary_name][0]:
                    attributes_dictionaries[dictionary_name][0].append(phrase)
                    attributes_dictionaries[dictionary_name][1].append(line[2][labels_list.index(lab)])

In [255]:
i = 1
for phrase, cls in zip(*attributes_dictionaries_1["object"]):
    if cls!=i:
        continue
    print(phrase)

домик
коттеджем
постройку
особняка
постройкой
строения
домом
постройка
зданием
строением
коттедж
дома
дому
изба
избой
здания
особняк
постройке
строение
постройки
домику
зданию
домиком
строению
особняком
здание
коттеджа
избе
коттеджу
избу
дом
домика
избы
особняку


In [256]:
for phrase, cls in zip(*attributes_dictionaries["object"]):
    if cls!=i:
        continue
    print(phrase)

дом
здание
изба
строение
постройка
коттедж
особняк
домик
здания
особняка
коттеджа
постройку
зданием
домиком
избы
строения
постройки
особняком
дома
зданию
дому
строению
коттеджем
домика
избу
коттеджу
домом
постройке
домику
особняку
постройкой
избе
избой
строением


In [192]:
model_args = ModelArgs(max_seq_length=256)

model = RepresentationModel(
    "bert", 
    "/s/ls4/groups/g0126/transformers_models/cointegrated/rubert-tiny2/",
    args=model_args
)

Some weights of the model checkpoint at /s/ls4/groups/g0126/transformers_models/cointegrated/rubert-tiny2/ were not used when initializing BertForTextRepresentation: ['cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForTextRepresentation from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTextRepresentation from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [257]:
import torch
state_dict = torch.load("/s/ls4/users/grartem/RL_robots/CommandClassifier/models/Prototype_pretrainedNormalizeModel/best/pytorch_model.bin")
old_keys=[]
for k in state_dict:
    old_keys.append(k)
for k in old_keys:
    state_dict["bert."+k] = state_dict[k]
    state_dict.pop(k)
model.model.load_state_dict(state_dict)

<All keys matched successfully>

In [258]:
# создаём эмбеддинги для словарей
attributes_embeddings = {}
for k in attributes_dictionaries.keys():
    attributes_embeddings[k] = np.zeros((len(attributes_dictionaries[k][0]), 312))
    word_embeddings = model.encode_sentences(attributes_dictionaries[k][0], combine_strategy=0)
    attributes_embeddings[k] = word_embeddings

In [259]:
test_df = pd.read_csv("../../Data/Interim/one_cmd_v4_05072022_low_split.csv")
test_phrases = test_df.loc[test_df["subset"]=="test", "x"]
test_df = test_df.loc[test_df["subset"]=="test", attributes_list]

In [260]:
from numpy import dot
from numpy.linalg import norm
#['action', 'direction', 'meters', 'degshours', 'object1', 'nearest', 'relation1', 'object2', 'relation2', 'object3', 'self', 'gaze']
predicted_actions = np.zeros((len(test_df), max(attributes_dictionaries["action"][1])+1))
predicted_attributes = np.zeros((len(test_df), test_df.shape[1]-1))
for i, (lab, dictionary_name) in enumerate(zip(labels_list, dictionaries_names_list)):
    phrases = []
    for sample in ParsedData:
        phrase = " ".join([x for x,y in zip(sample[1], sample[2]) if y==lab])
        phrases.append(phrase)
    word_embeddings = model.encode_sentences(phrases, combine_strategy=0)
    cos_sim = dot(word_embeddings, attributes_embeddings[dictionary_name].transpose())/ (np.expand_dims(norm(word_embeddings, axis=1), 1) * np.expand_dims(norm(attributes_embeddings[dictionary_name], axis=1), 0))
    
    #занулить близости для пустых фраз, чтоб не мешали
    emptyphraseFilter = np.array([True if p=="" else False for p in phrases])
    cos_sim[emptyphraseFilter] = 0
    if dictionary_name == 'action':
        for action_i in range(max(attributes_dictionaries["action"][1])+1):
            predicted_actions[:, action_i] = np.max(cos_sim[:,np.array(attributes_dictionaries[dictionary_name][1]) == action_i], axis=1)
    else:
        predicted_attributes[:,i-1] = np.array(attributes_dictionaries[dictionary_name][1])[np.argmax(cos_sim, axis=1)]

# оценки

In [261]:
possible_combinations = pd.read_csv("../../Data/Interim/possible_combinations_of_attributes_for_actions.csv")
possible_combinations_arr = possible_combinations.loc[:, attributes_list].values

In [262]:
predictions_3 = []
for i in range(len(predicted_actions)):
    fltr = np.ones(len(possible_combinations_arr), dtype=np.bool)
    for l_i, (lab, attribute_type) in enumerate(zip(labels_list[1:], attributes_list[1:])):
        if lab in ParsedData[i][2]:
            # и/или
            fltr |= possible_combinations[attribute_type]>0
        else:
            fltr &= possible_combinations[attribute_type]==0
    if sum(fltr)==0:
        raise ValueError("Предсказанная комбинация атрибутов не найдена среди возможных")
        # нужны вероятности для атрибутов
    possible_action = possible_combinations[fltr]["action"].values
    pred_action = possible_action[np.argmax(predicted_actions[i, possible_action])]
    
    active_attributes = possible_combinations[(possible_combinations["action"]==pred_action) & fltr].values[:,1:]
    active_attributes = active_attributes[np.argmax(np.sum(active_attributes, axis=1))]
    predictions_3.append(np.concatenate([[pred_action], predicted_attributes[i]*active_attributes]))
predictions_3 = np.array(predictions_3)

In [263]:
from RobotCommandClassifier import utils
from RobotCommandClassifier import *

In [243]:
print(test_phrases.iloc[0])
print(test_df.iloc[0])
print(ParsedData[0])
print(predictions_3[0])

постереги
action       0
direction    0
meters       0
degshours    0
object1      0
nearest      0
relation1    0
object2      0
relation2    0
object3      0
self         0
gaze         0
Name: 55828, dtype: int64
['постереги', ['постереги'], ['A'], [[[-3.46875, 18.328125, -2.654296875, -3.421875, -4.50390625, -0.9833984375, -3.685546875, -1.630859375, -1.5048828125, -3.06640625, -2.86328125, -3.08203125, -4.80859375], [-3.705078125, 16.234375, -3.908203125, -4.41796875, -4.6875, 3.451171875, -4.2734375, -3.701171875, -0.90478515625, -4.7578125, -3.615234375, -3.376953125, -5.15625]]]]
[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [188]:
result = utils.calculate_metrics_2(test_df, predicted_labels, display=True)

#result_avg = utils.calculate_metrics(test_y_df.iloc[:,1:], predictions_2[:,1:], config={
result_avg = utils.calculate_metrics(test_df, predicted_labels, config={
    "report_metrics": ["correct_samples_perc", "avg_macro_f1", "avg_acc", "class_acc", "class_macro_f1"]
})
for k, v in result_avg.items():
    print(np.round(v*100), "\t", k)

NameError: name 'predicted_labels' is not defined

In [264]:
from numpy import dot
from numpy.linalg import norm
def softmax(x):
    #e_x = x - np.expand_dims(np.max(x, axis=1), axis=1)
    #return e_x / np.expand_dims(e_x.sum(axis=1), axis=1)
    e_x = np.exp(x)
    return e_x / np.expand_dims(np.sum(e_x, axis=1), axis=1)

predicts_one_by_one = []
for s_i, sample in enumerate(ParsedData):
    #if test_df.iloc[s_i]["action"]!=0:
    #    continue
    #if s_i<500:
    #    continue
    #print(sample)
    # сначала извлекаем фразы из токенов для которых вероятность того, что они action больше порога
    predicted_actions_for_sample = np.zeros(max(attributes_dictionaries["action"][1])+1)
    sample_action_phrases = []
    current_phrase = []    
    for w_i in range(len(sample[1])):
        probs = sample[3][w_i]
        
        probs = np.max(softmax(np.array(probs)), axis=0)
        #probs = np.mean(softmax(np.array(probs)), axis=0)
        #probs = softmax(np.array(probs))[0,:]
        
        if probs[labels_list.index("A")+1] > 0.3:  # +1 Из-за того, что первым идёт O
            current_phrase.append(sample[1][w_i])
        else:
            if len(current_phrase)>0:
                sample_action_phrases.append(" ".join(current_phrase))
                current_phrase = []
    if len(current_phrase)>0:
        sample_action_phrases.append(" ".join(current_phrase))
    #print("define ACTION")
    #print(sample_action_phrases)
    if len(sample_action_phrases)==0:
        dummy = np.zeros(12)
        dummy[0] = 20
        predicts_one_by_one.append(dummy)
        continue
    # для собранных фраз экшенов подбираем близкие фразы из словаря экшенов
    phrases_embeddings = model.encode_sentences(sample_action_phrases, combine_strategy=0)
    cos_sim = dot(phrases_embeddings, attributes_embeddings["action"].transpose())/ (np.expand_dims(norm(phrases_embeddings, axis=1), 1) * np.expand_dims(norm(attributes_embeddings["action"], axis=1), 0))
    for action_i in range(max(attributes_dictionaries["action"][1])+1):
        predicted_actions_for_sample[action_i] = np.max(cos_sim[:,np.array(attributes_dictionaries["action"][1]) == action_i])
        #print(attribute_names["action"][action_i], np.max(cos_sim[:,np.array(attributes_dictionaries["action"][1]) == action_i]))
        for phrase_i in range(cos_sim.shape[0]):
            closest_phrase_id = np.argmax(cos_sim[phrase_i,np.array(attributes_dictionaries["action"][1]) == action_i])
            #print(np.array(attributes_dictionaries["action"][0])[np.array(attributes_dictionaries["action"][1]) == action_i][closest_phrase_id])
    probs_for_combinations = np.zeros_like(possible_combinations_arr, dtype=np.float32)
    probs_for_combinations[:,0] = np.take(predicted_actions_for_sample, possible_combinations_arr[:,0])
    
    
    for l_i, (lab, attribute_type) in enumerate(zip(labels_list[1:], attributes_list[1:])):
        #print("define", attribute_type)
        sample_attr_phrases = []
        current_phrase = []
        max_prob_for_label_appearence = 0
        for w_i in range(len(sample[1])):
            probs = sample[3][w_i]
            probs = np.max(softmax(np.array(probs)), axis=0)
            max_prob_for_label_appearence = max(max_prob_for_label_appearence, probs[labels_list.index(lab)+1])
        #print(np.round(max_prob_for_label_appearence))
        #print(possible_combinations_arr[:,l_i+1])
        probs_for_combinations[:,l_i+1] = np.take([1-max_prob_for_label_appearence, max_prob_for_label_appearence], possible_combinations_arr[:,l_i+1])
        #print(probs_for_combinations[:,l_i+1])
    best_template_i = np.argmax(np.sum(probs_for_combinations, axis=1))
    sample_predict_labels = possible_combinations.iloc[best_template_i].values.copy()
    for l_i, (lab, attribute_type) in enumerate(zip(labels_list[1:], dictionaries_names_list[1:])):
        if possible_combinations.iloc[best_template_i].values[l_i+1]==0:
            continue
        sample_attr_phrases = []
        current_phrase = []
        for w_i in range(len(sample[1])):
            probs = sample[3][w_i]
            probs = np.max(softmax(np.array(probs)), axis=0)
            if probs[labels_list.index(lab)+1] > 0.3:  # +1 Из-за того, что первым идёт O
                current_phrase.append(sample[1][w_i])
            else:
                if len(current_phrase)>0:
                    sample_attr_phrases.append(" ".join(current_phrase))
                    current_phrase = []
        if len(current_phrase)>0:
            sample_attr_phrases.append(" ".join(current_phrase))
        if len(sample_attr_phrases)>0:
            phrases_embeddings = model.encode_sentences(sample_attr_phrases, combine_strategy=0)
            cos_sim = dot(phrases_embeddings, attributes_embeddings[attribute_type].transpose())/ (np.expand_dims(norm(phrases_embeddings, axis=1), 1) * np.expand_dims(norm(attributes_embeddings[attribute_type], axis=1), 0))
            cos_sim = np.max(cos_sim, axis=0)
            sample_predict_labels[l_i+1] = np.array(attributes_dictionaries[attribute_type][1])[np.argmax(cos_sim)]
        else:
            sample_predict_labels[l_i+1] = 20
    #print("TRUE", test_df.iloc[s_i])
    #print("PREDICTED:", sample_predict_labels)    
    predicts_one_by_one.append(sample_predict_labels)
#    if s_i>=500:
#        break
    #print()
predicts_one_by_one = np.array(predicts_one_by_one)

In [189]:
for w_i in range(len(sample[1])):
    probs = sample[3][w_i]
    probs = np.max(softmax(np.array(probs)), axis=0)
    print(probs, probs[labels_list.index("A")+1])
    if probs[labels_list.index("A")+1] > 0.5:  # +1 Из-за того, что первым идёт O
        current_phrase.append(sample[1][w_i])
        print(current_phrase)
    else:
        if len(current_phrase)>0:
            sample_action_phrases.append(" ".join(current_phrase))
            current_phrase = []

[9.83174600e-10 4.85592405e-09 7.49422080e-10 1.06931239e-09
 7.56776507e-10 1.15394637e-09 5.08075711e-10 9.99999983e-01
 9.87022637e-10 1.61442025e-10 4.35920307e-09 3.33855612e-10
 9.36321218e-10] 4.855924053165388e-09
[1.22358056e-09 4.35282219e-09 9.49210394e-10 1.09895483e-09
 4.59008154e-10 7.46500361e-10 2.44253984e-10 9.99999984e-01
 1.14720474e-09 2.17243990e-10 4.54171586e-09 4.58112529e-10
 4.93406610e-10] 4.352822193133047e-09
[5.22238024e-09 1.41682020e-08 9.68076360e-09 2.11829079e-09
 1.46729715e-09 1.09895479e-09 1.42771931e-09 3.45853092e-09
 9.99999946e-01 4.44084160e-09 8.62708272e-09 1.11190892e-09
 1.13827708e-09] 1.416820200177976e-08
[9.99999990e-01 3.58875033e-10 4.32885921e-10 7.97755657e-10
 7.47848150e-11 3.60279630e-10 1.83985362e-11 1.51683390e-09
 1.11517129e-09 2.65653541e-10 3.87289826e-09 2.72485011e-10
 5.77978535e-10] 3.588750333119706e-10
[1.29253844e-08 9.18338879e-09 5.64668226e-09 6.08171611e-09
 4.79227699e-09 9.99989824e-01 1.28982385e-09 2.042

In [274]:
for i in range(len(predicts_one_by_one)):
    true_cortatest_df

,action,direction,meters,degshours,object1,nearest,relation1,object2,relation2,object3,self,gaze
55828,0,0,0,0,0,0,0,0,0,0,0,0
55829,1,0,0,0,0,0,0,0,0,0,0,0
55830,1,0,0,0,0,0,0,0,0,0,0,0
55831,1,0,0,0,0,0,0,0,0,0,0,0
55832,1,0,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...
57141,6,0,0,0,3,0,2,4,1,6,0,0
57142,6,0,0,0,3,0,2,4,1,6,0,0
57143,6,0,0,0,3,0,2,4,1,6,0,0
57144,6,0,0,0,3,0,2,4,1,6,0,0


In [22]:
result = utils.calculate_metrics_2(test_df, predicts_one_by_one, display=True)

#result_avg = utils.calculate_metrics(test_y_df.iloc[:,1:], predictions_2[:,1:], config={
result_avg = utils.calculate_metrics(test_df, predicts_one_by_one, config={
    "report_metrics": ["correct_samples_perc", "avg_macro_f1", "avg_acc", "class_acc", "class_macro_f1"]
})
for k, v in result_avg.items():
    print(np.round(v*100), "\t", k)

action
              precision    recall  f1-score   support

         0.0       0.62      0.30      0.41        33
         1.0       0.31      0.56      0.40        18
         2.0       0.32      0.11      0.16        56
         3.0       1.00      0.09      0.17        33
         5.0       0.00      0.00      0.00         0
         6.0       0.98      0.94      0.96      1178
         7.0       0.00      0.00      0.00         0
         8.0       0.00      0.00      0.00         0
         9.0       0.00      0.00      0.00         0
        10.0       0.00      0.00      0.00         0
        11.0       0.00      0.00      0.00         0
        12.0       0.00      0.00      0.00         0
        13.0       0.00      0.00      0.00         0
        20.0       0.00      0.00      0.00         0

    accuracy                           0.86      1318
   macro avg       0.23      0.14      0.15      1318
weighted avg       0.93      0.86      0.89      1318

direction
        

/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

              precision    recall  f1-score   support

         0.0       0.86      0.92      0.89       193
         1.0       0.94      0.75      0.83       550
         2.0       0.81      0.93      0.87       575
         3.0       0.00      0.00      0.00         0
         4.0       0.00      0.00      0.00         0
         5.0       0.00      0.00      0.00         0
        20.0       0.00      0.00      0.00         0

    accuracy                           0.85      1318
   macro avg       0.37      0.37      0.37      1318
weighted avg       0.87      0.85      0.85      1318

object2
              precision    recall  f1-score   support

         0.0       0.86      0.92      0.89       193
         1.0       0.88      0.86      0.87       195
         2.0       0.64      0.58      0.61       155
         3.0       0.80      0.49      0.61       205
         4.0       0.75      0.88      0.81       225
         5.0       0.77      0.70      0.73       190
         6.0    

/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

In [202]:
result = utils.calculate_metrics_2(test_df, predicts_one_by_one, display=True)

#result_avg = utils.calculate_metrics(test_y_df.iloc[:,1:], predictions_2[:,1:], config={
result_avg = utils.calculate_metrics(test_df, predicts_one_by_one, config={
    "report_metrics": ["correct_samples_perc", "avg_macro_f1", "avg_acc", "class_acc", "class_macro_f1"]
})
for k, v in result_avg.items():
    print(np.round(v*100), "\t", k)

action
              precision    recall  f1-score   support

         0.0       0.93      0.42      0.58        33
         1.0       0.33      0.56      0.42        18
         2.0       0.30      0.11      0.16        56
         3.0       1.00      0.06      0.11        33
         5.0       0.00      0.00      0.00         0
         6.0       0.98      0.96      0.97      1178
         7.0       0.00      0.00      0.00         0
         8.0       0.00      0.00      0.00         0
         9.0       0.00      0.00      0.00         0
        10.0       0.00      0.00      0.00         0
        11.0       0.00      0.00      0.00         0
        12.0       0.00      0.00      0.00         0
        13.0       0.00      0.00      0.00         0
        20.0       0.00      0.00      0.00         0

    accuracy                           0.88      1318
   macro avg       0.25      0.15      0.16      1318
weighted avg       0.94      0.88      0.90      1318

direction
        

/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

45.0 	 correct_samples_perc
88.0 	 [action]_acc
16.0 	 [action]_macrof1
95.0 	 [direction]_acc
38.0 	 [direction]_macrof1
100.0 	 [meters]_acc
100.0 	 [meters]_macrof1
100.0 	 [degshours]_acc
50.0 	 [degshours]_macrof1
77.0 	 [object1]_acc
33.0 	 [object1]_macrof1
99.0 	 [nearest]_acc
50.0 	 [nearest]_macrof1
84.0 	 [relation1]_acc
44.0 	 [relation1]_macrof1
81.0 	 [object2]_acc
36.0 	 [object2]_macrof1
86.0 	 [relation2]_acc
45.0 	 [relation2]_macrof1
82.0 	 [object3]_acc
41.0 	 [object3]_macrof1
99.0 	 [self]_acc
20.0 	 [self]_macrof1
98.0 	 [gaze]_acc
49.0 	 [gaze]_macrof1
43.0 	 avg_macro_f1
91.0 	 avg_acc


/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

In [217]:
result = utils.calculate_metrics_2(test_df, predicts_one_by_one, display=True)

#result_avg = utils.calculate_metrics(test_y_df.iloc[:,1:], predictions_2[:,1:], config={
result_avg = utils.calculate_metrics(test_df, predicts_one_by_one, config={
    "report_metrics": ["correct_samples_perc", "avg_macro_f1", "avg_acc", "class_acc", "class_macro_f1"]
})
for k, v in result_avg.items():
    print(np.round(v*100), "\t", k)

action
              precision    recall  f1-score   support

         0.0       1.00      0.36      0.53        33
         1.0       0.31      0.56      0.40        18
         2.0       0.30      0.11      0.16        56
         3.0       1.00      0.06      0.11        33
         5.0       0.00      0.00      0.00         0
         6.0       0.98      0.97      0.98      1178
         7.0       0.00      0.00      0.00         0
         8.0       0.00      0.00      0.00         0
         9.0       0.00      0.00      0.00         0
        10.0       0.00      0.00      0.00         0
        11.0       0.00      0.00      0.00         0
        12.0       0.00      0.00      0.00         0
        13.0       0.00      0.00      0.00         0
        20.0       0.00      0.00      0.00         0

    accuracy                           0.89      1318
   macro avg       0.26      0.15      0.16      1318
weighted avg       0.94      0.89      0.90      1318

direction
        

/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

37.0 	 correct_samples_perc
89.0 	 [action]_acc
16.0 	 [action]_macrof1
95.0 	 [direction]_acc
38.0 	 [direction]_macrof1
100.0 	 [meters]_acc
100.0 	 [meters]_macrof1
100.0 	 [degshours]_acc
50.0 	 [degshours]_macrof1
68.0 	 [object1]_acc
30.0 	 [object1]_macrof1
99.0 	 [nearest]_acc
50.0 	 [nearest]_macrof1
85.0 	 [relation1]_acc
45.0 	 [relation1]_macrof1
76.0 	 [object2]_acc
33.0 	 [object2]_macrof1
86.0 	 [relation2]_acc
45.0 	 [relation2]_macrof1
79.0 	 [object3]_acc
42.0 	 [object3]_macrof1
99.0 	 [self]_acc
20.0 	 [self]_macrof1
98.0 	 [gaze]_acc
49.0 	 [gaze]_macrof1
43.0 	 avg_macro_f1
90.0 	 avg_acc


/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

In [265]:
result = utils.calculate_metrics_2(test_df, predicts_one_by_one, display=True)

#result_avg = utils.calculate_metrics(test_y_df.iloc[:,1:], predictions_2[:,1:], config={
result_avg = utils.calculate_metrics(test_df, predicts_one_by_one, config={
    "report_metrics": ["correct_samples_perc", "avg_macro_f1", "avg_acc", "class_acc", "class_macro_f1"]
})
for k, v in result_avg.items():
    print(np.round(v*100), "\t", k)

action
              precision    recall  f1-score   support

         0.0       1.00      0.36      0.53        33
         1.0       0.31      0.56      0.40        18
         2.0       0.30      0.11      0.16        56
         3.0       1.00      0.06      0.11        33
         5.0       0.00      0.00      0.00         0
         6.0       0.98      0.97      0.98      1178
         7.0       0.00      0.00      0.00         0
         8.0       0.00      0.00      0.00         0
         9.0       0.00      0.00      0.00         0
        10.0       0.00      0.00      0.00         0
        11.0       0.00      0.00      0.00         0
        12.0       0.00      0.00      0.00         0
        13.0       0.00      0.00      0.00         0
        20.0       0.00      0.00      0.00         0

    accuracy                           0.89      1318
   macro avg       0.26      0.15      0.16      1318
weighted avg       0.94      0.89      0.90      1318

direction
        

/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

46.0 	 correct_samples_perc
89.0 	 [action]_acc
16.0 	 [action]_macrof1
95.0 	 [direction]_acc
38.0 	 [direction]_macrof1
100.0 	 [meters]_acc
100.0 	 [meters]_macrof1
100.0 	 [degshours]_acc
50.0 	 [degshours]_macrof1
77.0 	 [object1]_acc
33.0 	 [object1]_macrof1
99.0 	 [nearest]_acc
50.0 	 [nearest]_macrof1
85.0 	 [relation1]_acc
45.0 	 [relation1]_macrof1
81.0 	 [object2]_acc
36.0 	 [object2]_macrof1
86.0 	 [relation2]_acc
45.0 	 [relation2]_macrof1
82.0 	 [object3]_acc
41.0 	 [object3]_macrof1
99.0 	 [self]_acc
20.0 	 [self]_macrof1
98.0 	 [gaze]_acc
49.0 	 [gaze]_macrof1
43.0 	 avg_macro_f1
91.0 	 avg_acc


/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/s/ls4/users/grartem/anaconda3/envs/simptr/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(resu

In [207]:
attribute_names["action"]

['patrol',
 'stop',
 'move_dir',
 'rotate_dir',
 'move_on',
 'rotate_on',
 'move_to',
 'find',
 'go_around',
 'monitor',
 'rotate_to',
 'pause',
 'continue',
 'analyze']

In [203]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(test_df["action"], predicts_one_by_one[:,0]))

[[  14    2    0    0    0    8    0    0    3    0    4    0    2    0]
 [   0   10    0    0    0    6    0    0    0    0    2    0    0    0]
 [   1   17    6    0    2    6    0    0    0    0   16    7    0    1]
 [   0    1   14    2    2    3    0    0    0    1    0    0    0   10]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0 1132   14   16    0    0    0    0    0   16]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0

In [267]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(test_df["action"], predicts_one_by_one[:,0]))

[[  12    4    0    0    0    8    0    0    2    0    4    0    3    0]
 [   0   10    0    0    0    6    0    0    0    0    2    0    0    0]
 [   0   17    6    0    2    6    0    0    0    0   17    7    0    1]
 [   0    1   14    2    2    3    0    0    0    1    0    0    0   10]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0 1148   13    1    0    0    0    0    0   16]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0    0    0    0    0]
 [   0    0    0    0    0    0    0    0    0    0

In [93]:
test_df.iloc[500]

action       6
direction    0
meters       0
degshours    0
object1      1
nearest      0
relation1    1
object2      5
relation2    1
object3      4
self         0
gaze         0
Name: 56328, dtype: int64